<picture>
  <source media="(prefers-color-scheme: dark)" srcset="https://assets.vespa.ai/logos/Vespa-logo-green-RGB.svg">
  <source media="(prefers-color-scheme: light)" srcset="https://assets.vespa.ai/logos/Vespa-logo-dark-RGB.svg">
  <img alt="#Vespa" width="200" src="https://assets.vespa.ai/logos/Vespa-logo-dark-RGB.svg" style="margin-bottom: 25px;">
</picture>

# Billion-scale vector search with Cohere binary embeddings in Vespa

Cohere just released a new embedding API with support for binary and `int8` vectors. Read the announcement
in the blog post: [Cohere int8 & binary Embeddings - Scale Your Vector Database to Large Datasets](https://cohere.com/blog/int8-binary-embeddings).

> We are excited to announce that Cohere Embed is the first embedding model that natively supports int8 and binary embeddings.

This is huge because:

- Binarization reduces the storage (disk/memory) footprint from 1024 floats (4096 bytes) per vector to 128 bytes.
- Faster distance calculations using [hamming](https://docs.vespa.ai/en/reference/schema-reference.html#distance-metric) distance that
  Vespa natively supports for bits packed into `int8` tensor cells. More on [hamming distance in Vespa](https://docs.vespa.ai/en/reference/schema-reference.html#hamming).
- Multiple vector representations allow for coarse retrieval in hamming space and subsequent phases using higher-resolution representations.
- Drastically reduces the deployment due to tiered storage economics.

Vespa supports `hamming` distance with and without [HNSW indexing](https://docs.vespa.ai/en/approximate-nn-hnsw.html).

For those wanting to learn more about binary vectors, we recommend our 2021 blog series on [Billion-scale vector search with Vespa](https://blog.vespa.ai/billion-scale-knn/)
and [Billion-scale vector search with Vespa - part two](https://blog.vespa.ai/billion-scale-knn-part-two/).

This notebook demonstrates using the Cohere embeddings with a coarse-to-fine search and re-ranking pipeline that reduces costs, but
offers the same retrieval (nDCG) accuracy.

- The packed binary vector representation is stored in memory, with an optional HNSW index using hamming distance.
- The `int8` vector representation is stored on disk using Vespa's [paged](https://docs.vespa.ai/en/attributes.html#paged-attributes) option.

At query time:

- Retrieve in hamming space (1000 candidates) as the coarse-level search using the compact binary representation.
- Re-rank by using a dot product between the float version of the query vector (1024 dims) against an unpacked float version of the binary embedding (also 1024 dims)
- A re-ranking phase using the 1024 dimensional int8 representations. This stage pages the vector data from the disk using Vespa's [paged](https://docs.vespa.ai/en/attributes.html#paged-attributes) option (unless it is already cached).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vespa-engine/pyvespa/blob/master/docs/sphinx/source/examples/billion-scale-vector-search-with-cohere-embeddings-cloud.ipynb)

Install the dependencies:


In [ ]:
!pip3 install -U pyvespa cohere==4.57 vespacli

## Examining the Cohere embeddings

Let us check out the Cohere embedding API and how we can obtain vector embeddings with different precisions for the same text input (without
additional cost). See also [Cohere embed API doc](https://docs.cohere.com/docs/embed-api).


In [3]:
import cohere

# Make sure that the environment variable CO_API_KEY is set to your API key
co = cohere.Client()

### Some sample documents

Define a few sample documents that we want to embed


In [4]:
documents = [
    "Alan Turing  was an English mathematician, computer scientist, logician, cryptanalyst, philosopher and theoretical biologist.",
    "Albert Einstein was a German-born theoretical physicist who is widely held to be one of the greatest and most influential scientists of all time.",
    "Isaac Newton was an English polymath active as a mathematician, physicist, astronomer, alchemist, theologian, and author who was described in his time as a natural philosopher.",
    "Marie Curie was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity",
]

In [5]:
# Compute the  embeddings of our sample documents.
# Set input_type to "search_document" and embedding_types to "binary" and "int8"
embeddings = co.embed(
    texts=documents,
    model="embed-english-v3.0",
    input_type="search_document",
    embedding_types=["binary", "int8"],
)

In [6]:
print(embeddings)

cohere.Embeddings {
	response_type: embeddings_by_type
	embeddings: cohere.EmbeddingsByType {
	float: None
	int8: [[-23, -22, -52, 18, -42, -48, 2, -8, 6, 44, 73, 9, 3, -44, -25, 15, 19, 3, 18, -19, 6, 17, 0, -62, -14, 46, -8, -14, 20, 22, 10, -40, 10, 48, -20, 40, -8, 8, 29, 0, -27, 11, -39, -28, -93, 33, -89, 4, 15, -41, -12, -2, 7, -23, -15, -21, 47, -9, 88, -107, -91, -50, 65, 27, 5, 5, 52, 27, -15, -4, 14, -7, 6, -1, -17, 13, 17, 74, 26, -9, -4, -1, 56, -15, -7, 6, -17, -25, -23, -38, -38, 78, -61, -27, -53, -20, -3, -8, -9, -18, 9, -24, 14, -13, -40, 90, 40, 24, -48, -7, -11, -116, 36, -56, -15, -1, -6, 31, 31, 8, 44, 80, 36, -35, -24, -13, -36, -64, 44, -11, -35, 46, -43, -68, -40, 12, 32, -8, -1, 58, -9, -4, 49, 3, 9, 44, 45, -33, -52, -25, -53, 27, -67, 22, 33, 29, -32, 36, 37, 83, -17, 19, 66, -17, 4, -57, -57, 20, 19, -20, -3, 18, 43, -16, -8, 29, -45, -39, -42, 121, 73, -49, -128, 127, -19, 41, -10, 55, 38, 13, -66, 1, -52, -35, 59, 6, -60, -35, 20, -11, -20, 58, -50, 27, -

As we can see from the above, we got multiple vector representations for the same input strings.


In [54]:
print(
    "int8 dimensionality: {}, binary dimensionality {}".format(
        len(embeddings.embeddings.int8[0]), len(embeddings.embeddings.binary[0])
    )
)

int8 dimensionality: 1024, binary dimensionality 128


## Defining the Vespa application

First, we define a [Vespa schema](https://docs.vespa.ai/en/schemas.html) with the fields we want to store and their type.

Notice the `binary_vector` field that defines an indexed (dense) Vespa tensor with the dimension name `x[128]`.

Indexing specifies `index` which means that Vespa will build HNSW graph for searching this vector field.

Also, notice the configuration of the [distance-metric](https://docs.vespa.ai/en/reference/schema-reference.html#distance-metric).

We also want to store the `int8_vector` on disk; we use `paged` to signalize this.


In [8]:
from vespa.package import Schema, Document, Field, FieldSet

my_schema = Schema(
    name="doc",
    mode="index",
    document=Document(
        fields=[
            Field(name="doc_id", type="string", indexing=["summary"]),
            Field(
                name="text",
                type="string",
                indexing=["summary", "index"],
                index="enable-bm25",
            ),
            Field(
                name="binary_vector",
                type="tensor<int8>(x[128])",
                indexing=["attribute", "index"],
                attribute=["distance-metric: hamming"],
            ),
            Field(
                name="int8_vector",
                type="tensor<int8>(x[1024])",
                indexing=["attribute"],
                attribute=["paged"],
            ),
        ]
    ),
    fieldsets=[FieldSet(name="default", fields=["text"])],
)

We must add the schema to a Vespa [application package](https://docs.vespa.ai/en/application-packages.html).
This consists of configuration files, schemas, models, and possibly even custom code (plugins).


In [9]:
from vespa.package import ApplicationPackage

vespa_app_name = "coherebillion"
vespa_application_package = ApplicationPackage(name=vespa_app_name, schema=[my_schema])

In the last step, we configure [ranking](https://docs.vespa.ai/en/ranking.html) by adding `rank-profile`'s to the schema.

`unpack_bits`, unpacks the binary representation into a 1024-dimensional float vector [doc](https://docs.vespa.ai/en/reference/ranking-expressions.html#unpack-bits).

We define three tensor inputs that we intend to send with the query request.


In [10]:
from vespa.package import RankProfile, FirstPhaseRanking, SecondPhaseRanking, Function


rerank = RankProfile(
    name="rerank",
    inputs=[
        ("query(q_binary)", "tensor<int8>(x[128])"),
        ("query(q_full)", "tensor<float>(x[1024])"),
        ("query(q_int8)", "tensor<int8>(x[1024])"),
    ],
    functions=[
        Function(  # this returns a tensor<float>(x[1024]) with values -1 or 1
            name="unpack_binary_representation",
            expression="2*unpack_bits(attribute(binary_vector)) -1",
        )
    ],
    first_phase=FirstPhaseRanking(
        expression="sum(query(q_full)*unpack_binary_representation )"  # phase 1 ranking using the float query and the unpacked float version of the binary_vector
    ),
    second_phase=SecondPhaseRanking(
        expression="cosine_similarity(query(q_int8),attribute(int8_vector),x)",  # phase 2 using the int8 vector representations
        rerank_count=30,  # number of hits to rerank, upper bound on number of random IO operations
    ),
    match_features=[
        "distance(field, binary_vector)",
        "closeness(field, binary_vector)",
        "firstPhase",
    ],
)
my_schema.add_rank_profile(rerank)

## Deploy the application to Vespa Cloud

With the configured application, we can deploy it to [Vespa Cloud](https://cloud.vespa.ai/en/).

To deploy the application to Vespa Cloud we need to create a tenant in the Vespa Cloud:

Create a tenant at [console.vespa-cloud.com](https://console.vespa-cloud.com/) (unless you already have one).
This step requires a Google or GitHub account, and will start your [free trial](https://cloud.vespa.ai/en/free-trial).

Make note of the tenant name, it is used in the next steps.

> Note: Deployments to dev and perf expire after 7 days of inactivity, i.e., 7 days after running deploy. This applies to all plans, not only the Free Trial. Use the Vespa Console to extend the expiry period, or redeploy the application to add 7 more days.


In [39]:
from vespa.deployment import VespaCloud
import os

# Replace with your tenant name from the Vespa Cloud Console
tenant_name = "vespa-team"

# Key is only used for CI/CD. Can be removed if logging in interactively
key = os.getenv("VESPA_TEAM_API_KEY", None)
if key is not None:
    key = key.replace(r"\n", "\n")  # To parse key correctly

vespa_cloud = VespaCloud(
    tenant=tenant_name,
    application=vespa_app_name,
    key_content=key,  # Key is only used for CI/CD. Can be removed if logging in interactively
    application_package=vespa_application_package,
)

Now deploy the app to Vespa Cloud dev zone.

The first deployment typically takes 2 minutes until the endpoint is up.


In [ ]:
from vespa.application import Vespa

app: Vespa = vespa_cloud.deploy()

## Feed our sample documents and their binary embedding representation

With few documents, we use the synchronous API. Read more in [reads and writes](https://pyvespa.readthedocs.io/en/latest/reads-writes.html).


In [41]:
for i, doc in enumerate(documents):
    response = app.feed_data_point(
        schema="doc",
        data_id=str(i),
        fields={
            "doc_id": str(i),
            "text": doc,
            "binary_vector": embeddings.embeddings.binary[i],
            "int8_vector": embeddings.embeddings.int8[i],
        },
    )
    assert response.is_successful()

### Querying data

Read more about querying Vespa in:

- [Vespa Query API](https://docs.vespa.ai/en/query-api.html)
- [Vespa Query API reference](https://docs.vespa.ai/en/reference/query-api-reference.html)
- [Vespa Query Language API (YQL)](https://docs.vespa.ai/en/query-language.html)
- [Practical Nearest Neighbor Search Guide](https://docs.vespa.ai/en/nearest-neighbor-search-guide.html)

We now need to invoke the embed API again to embed the query text; we ask for all three representations:


In [30]:
query = "Who discovered x-ray?"

# Make sure to set input_type="search_query" when getting the embeddings for the query.
# We ask for 3 versions (float, binary, and int8) of the embeddings.
query_emb = co.embed(
    [query],
    model="embed-english-v3.0",
    input_type="search_query",
    embedding_types=["float", "binary", "int8"],
)

In [ ]:
print(query_emb)

Now, we use Vespa's [nearestNeighbor](https://docs.vespa.ai/en/reference/query-language-reference.html#nearestneighbor) query operator
to expose up to 1000 hits to ranking using the configured distance-metric (hamming distance).

This is the retrieve logic, or phase-0 search as it only uses the hamming distance. See [phased ranking](https://docs.vespa.ai/en/phased-ranking.html) for more
on phased ranking pipelines.

The hits that are near in hamming space, are exposed to the flexibility of the Vespa ranking framework:

- the first-phase uses the unpacked version of the binary vector and computes the dot product against the float query version
- The second phase and final phase re-ranks the 30 best from the the previous phase, here using cosine similarity between the int8 embedding representations


In [47]:
response = app.query(
    yql="select * from doc where {targetHits:1000}nearestNeighbor(binary_vector,q_binary)",
    ranking="rerank",
    body={
        "input.query(q_binary)": query_emb.embeddings.binary[0],
        "input.query(q_full)": query_emb.embeddings.float[0],
        "input.query(q_int8)": query_emb.embeddings.int8[0],
    },
)
assert response.is_successful()

In [48]:
response.hits

[{'id': 'id:doc:doc::3',
  'relevance': 0.45650564242263414,
  'source': 'cohere_content',
  'fields': {'matchfeatures': {'closeness(field,binary_vector)': 0.0030303030303030303,
    'distance(field,binary_vector)': 329.0,
    'firstPhase': 4.905200004577637},
   'sddocname': 'doc',
   'documentid': 'id:doc:doc::3',
   'doc_id': '3',
   'text': 'Marie Curie was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity'}},
 {'id': 'id:doc:doc::1',
  'relevance': 0.337421116422118,
  'source': 'cohere_content',
  'fields': {'matchfeatures': {'closeness(field,binary_vector)': 0.002544529262086514,
    'distance(field,binary_vector)': 391.99999999999994,
    'firstPhase': 3.7868080139160156},
   'sddocname': 'doc',
   'documentid': 'id:doc:doc::1',
   'doc_id': '1',
   'text': 'Albert Einstein was a German-born theoretical physicist who is widely held to be one of the greatest and most influential scientists of all time.'}},
 {'id': 'id:doc:do

The `relevance` is the cosine similarity between the int8 vector representations calculated in the second-phase. Note also that we return the `hamming` distance
and the firstPhase score which is the query, unpacked binary dot product.


## Conclusions

These new Cohere binary embeddings are a huge step forward for cost-efficient vector search at scale and integrate perfectly
with Vespa features for building out vector search at scale.

Storing the `int8` vector representation on disk using the paged attribute option enables phased retrieval and ranking close to the data.
First, one can use the compact in-memory binary representation for the coarse-level search to efficiently find a limited number of candidates.
Then, the candidates from the coarse search can be re-scored and re-ranked using a more advanced scoring function using a finer resolution.

### Clean up

We can now delete the cloud instance:


In [ ]:
vespa_cloud.delete()